In [1]:
import numpy as np
import pandas as pd
from sklearn.compose import make_column_selector

In [3]:
df = pd.read_csv("german.csv", sep= ",")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   checking_balance      1000 non-null   object
 1   months_loan_duration  1000 non-null   int64 
 2   credit_history        1000 non-null   object
 3   purpose               1000 non-null   object
 4   amount                1000 non-null   int64 
 5   savings_balance       1000 non-null   object
 6   employment_length     1000 non-null   object
 7   installment_rate      1000 non-null   int64 
 8   personal_status       1000 non-null   object
 9   other_debtors         1000 non-null   object
 10  residence_history     1000 non-null   int64 
 11  property              1000 non-null   object
 12  age                   1000 non-null   int64 
 13  installment_plan      1000 non-null   object
 14  housing               1000 non-null   object
 15  existing_credits      1000 non-null   i

In [4]:
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [5]:
y = df["default"]
X = df.copy()
X.drop("default", axis=1, inplace=True)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
(X_train, X_test, y_train, y_test,) = train_test_split(
        X,
        y,
        test_size=100,
        random_state=123,
    )

In [9]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [10]:
# Cree un objeto ColumnTransformer que aplique OneHotEncoder a las columnas
    # tipo texto. Use make_column_selector para seleccionar las columnas. Las
    # columnas numéricas no deben ser transformadas.
columnTransformer = make_column_transformer(
        (
            OneHotEncoder(),
            make_column_selector(dtype_include=object),
        ),
        remainder='passthrough',
    )

In [12]:
# Cree un pipeline que contenga el columnTransformer y el modelo SVC.
pipeline = Pipeline(
        steps=[
            ("ct", columnTransformer),
            ("svc", SVC()),
        ],
    )

In [13]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002B6CF35D090>)])),
                ('svc', SVC())])

In [14]:
from sklearn.metrics import confusion_matrix

In [15]:
cfm_train = confusion_matrix(
        y_true=y_train,
        y_pred=pipeline.predict(X_train),
    )

cfm_train

array([[626,   8],
       [250,  16]], dtype=int64)

In [17]:
cfm_test = confusion_matrix(
        y_true=y_test,
        y_pred=pipeline.predict(X_test),
    )

cfm_test 

array([[66,  0],
       [30,  4]], dtype=int64)